In [1]:
# !pip install deeprobust
# !conda install pytorch torchvision torchaudio -c pytorch
import torch
# print(torch.__version__)
# !pip install torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
# !pip install torch-geometric
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

from networkx.generators.random_graphs import erdos_renyi_graph
from networkx.generators.random_graphs import barabasi_albert_graph
from networkx.generators.community import stochastic_block_model
from networkx.generators.random_graphs import watts_strogatz_graph
from networkx.generators.community import random_partition_graph

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import math
from tqdm import tqdm
import seaborn as sns
from sklearn.decomposition import FactorAnalysis

import random
from torch.nn import Linear,Sigmoid
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, BatchNorm, GraphConv
from torch_geometric.nn import global_mean_pool
from random import sample

/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_44518/819003294.py:10: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import os
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as sp
import torch
from torch import Tensor
import torch_geometric
from torch_geometric.utils import to_networkx
from torch_geometric.datasets import Planetoid
import networkx as nx
from networkx.algorithms import community
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_dir = "./data"
os.makedirs(data_dir, exist_ok=True)

In [3]:
import numpy
import torch
%load_ext autoreload
%autoreload 2

%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))



from networkx.generators.random_graphs import erdos_renyi_graph
from networkx.generators.random_graphs import barabasi_albert_graph
from networkx.generators.community import stochastic_block_model
from networkx.generators.random_graphs import watts_strogatz_graph
from networkx.generators.community import random_partition_graph
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import math
from tqdm import tqdm
import seaborn as sns
from sklearn.decomposition import FactorAnalysis
import random

from random import sample
from scipy.sparse import csr_matrix
from scipy.sparse import csgraph
from scipy.sparse.linalg import inv

from scipy.sparse import random
from scipy.sparse.linalg import norm

from scipy.sparse import random
from scipy.stats import rv_continuous

from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_dense_adj
import math
from tqdm import tqdm
import seaborn as sns
from sklearn.decomposition import FactorAnalysis
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_44518/1785052457.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [4]:
from random import sample

In [5]:
# from deeprobust.graph.data import Dataset

In [6]:
from scipy.sparse import csr_matrix
from scipy.sparse import csgraph
from scipy.sparse.linalg import inv

In [7]:
import os
os.getcwd()
dataset = os.path.join(os.getcwd(),'PubMed')
dataset

'/Users/prakashpal/Desktop/Sandip Sir/PubMed'

In [8]:
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_dense_adj




dataset= Planetoid(root=dataset, name='PubMed')
print(dataset[0])
adj = to_dense_adj(dataset[0].edge_index)
adj = adj[0]
labels = dataset[0].y
labels = labels.numpy()

X = dataset[0].x
X = X.to_dense()
N = X.shape[0]
NO_OF_CLASSES =  len(set(np.array(dataset[0].y)))

print(X.shape, adj.shape)

nn = int(1*N)
X = X[:nn,:]
adj = adj[:nn,:nn]
labels = labels[:nn]
print(X.shape,adj.shape)

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])
torch.Size([19717, 500]) torch.Size([19717, 19717])
torch.Size([19717, 500]) torch.Size([19717, 19717])


In [9]:
Data = dataset[0]
Data

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])

In [10]:
import torch_geometric.transforms as T

split = T.RandomLinkSplit(
    num_val=0.01,
    num_test=0.02,
    is_undirected=True,
    add_negative_train_samples=False,
    neg_sampling_ratio=1.0,
)
tra, va, tes = split(Data)

In [11]:
tra

Data(x=[19717, 500], edge_index=[2, 85990], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717], edge_label=[42995], edge_label_index=[2, 42995])

In [12]:
tes

Data(x=[19717, 500], edge_index=[2, 86876], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717], edge_label=[1772], edge_label_index=[2, 1772])

In [13]:
va

Data(x=[19717, 500], edge_index=[2, 85990], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717], edge_label=[886], edge_label_index=[2, 886])

In [14]:
def get_laplacian(adj):
    b=torch.ones(adj.shape[0])
    return torch.diag(adj@b)-adj

theta = get_laplacian(adj)
print(theta.shape)


torch.Size([19717, 19717])


In [15]:

features = X.numpy()
NO_OF_NODES = X.shape[0]
print(features.shape)
print(NO_OF_CLASSES,NO_OF_NODES)

(19717, 500)
3 19717


In [16]:
def convertScipyToTensor(coo):
  try:
    coo = coo.tocoo()
  except:
    coo = coo
  values = coo.data
  indices = np.vstack((coo.row, coo.col))

  i = torch.LongTensor(indices)
  v = torch.FloatTensor(values)
  shape = coo.shape

  return torch.sparse.FloatTensor(i, v, torch.Size(shape))

In [17]:
from scipy.sparse import random
from scipy.sparse.linalg import norm
from scipy.sparse import csr_matrix

p = X.shape[0]
k = int(p*0.5)
n = X.shape[1]
lambda_param = 100
beta_param = 0.1
gamma_param = 100
lr = 1e-5
thresh = 1e-10

from scipy.sparse import random
from scipy.stats import rv_continuous
class CustomDistribution(rv_continuous):
    def _rvs(self,  size=None, random_state=None):
        return random_state.standard_normal(size)
temp = CustomDistribution(seed=1)
temp2 = temp()  # get a frozen version of the distribution
X_tilde = random(k, n, density=0.25, random_state=1, data_rvs=temp2.rvs)
C = random(p, k, density=0.25, random_state=1, data_rvs=temp2.rvs)



In [18]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv


class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(X.shape[1], 64)
        self.conv2 = GCNConv(64, NO_OF_CLASSES)

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()

    def forward(self, x, edge_index):

        #print("Checking 1: x", x.shape, "Edge index:", edge_index.shape)
        x = self.conv1(x, edge_index)
        #print("Checking 2: convolution done, new x:", x.shape)
        x = F.relu(x)
        #print("Checking 3: x", x.shape, "training:", self.training)
        x = F.dropout(x, training=self.training)
        #print("Checking 4: dropout done new x", x.shape, "Edge index:", edge_index.shape)
        x = self.conv2(x, edge_index)
        #print("Checking 5: x", x.shape)

        return F.log_softmax(x, dim=1)



In [19]:
features.shape

(19717, 500)

In [20]:
def feature(feat):
  graph = feat
  print("feat:",graph)
  print("------------------",type(feat))
  # type(feat)

In [21]:
X.shape

torch.Size([19717, 500])

In [22]:
def experiment_sparsity_2(lambda_param,beta_param,gamma_param,C,theta,X):
      p = X.shape[0]
      k = int(p*0.5)
      n = X.shape[1]
      ones = csr_matrix(np.ones((k,k)))
      ones = convertScipyToTensor(ones)
      ones = ones.to_dense()
      J = np.outer(np.ones(k), np.ones(k))/k
      J = csr_matrix(J)
      J = convertScipyToTensor(J)
      J = J.to_dense()
      zeros = csr_matrix(np.zeros((p,k)))
      zeros = convertScipyToTensor(zeros)
      zeros = zeros.to_dense()
      C = convertScipyToTensor(C)
      C = C.to_dense()
      eye = torch.eye(k)
      try:
        theta = convertScipyToTensor(theta)
      except:
        theta = theta
      try:
        X = convertScipyToTensor(X)
        X = X.to_dense()
      except:
        X = X

      if(torch.cuda.is_available()):
        print("yes")
#         X_tilde = X_tilde.cuda()
        C = C.cuda()
        theta = theta.cuda()
        X = X.cuda()
        J = J.cuda()
        zeros = zeros.cuda()
        ones = ones.cuda()
        eye = eye.cuda()

      def update(C,i):
          global L
          thetaC = theta@C
          CT = torch.transpose(C,0,1)
          t1 = CT@thetaC + J
          term_bracket = torch.linalg.pinv(t1)
          L = 1/k
          t1 = -2*gamma_param*(thetaC@term_bracket)
          t4 = lambda_param*(C@ones)
          t5 = 2*beta_param*(thetaC@CT@thetaC)
          T2=(t1+t4+t5)/L
          Cnew = (C-T2).maximum(zeros)
          Cnew[Cnew<thresh] = thresh
          for i in range(len(Cnew)):
              Cnew[i] = Cnew[i]/torch.linalg.norm(Cnew[i],1)
          # print("Cnew",Cnew)
          return Cnew

      for i in tqdm(range(20)):   #update C only 21
          C = update(C,i)

      feat = torch.linalg.pinv(C)@X
      # feature(feat)
      print("shape of feat", feat.shape)

      return feat,C



In [23]:
import torch
import torchvision
from sklearn.metrics import roc_auc_score
from torch_geometric.utils import negative_sampling


class Net1(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(
            dim=-1
        )  # product of a pair of nodes on each edge

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()


def train_link_predictor(
    model, train_data, val_data, optimizer, criterion, n_epochs=300
):

    for epoch in range(1, n_epochs + 1):

        model.train()
        optimizer.zero_grad()
        z = model.encode(train_data.x, train_data.edge_index)

        # sampling training negatives for every training epoch
        neg_edge_index = negative_sampling(
            edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
            num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

        edge_label_index = torch.cat(
            [train_data.edge_label_index, neg_edge_index],
            dim=-1,
        )
        edge_label = torch.cat([
            train_data.edge_label,
            train_data.edge_label.new_zeros(neg_edge_index.size(1))
        ], dim=0)

        out = model.decode(z, edge_label_index).view(-1)
        loss = criterion(out, edge_label)
        loss.backward()
        optimizer.step()

        val_auc = eval_link_predictor(model, val_data)

        if epoch % 100 == 0:
            print(f"Epoch: {epoch:03d}, Train Loss: {loss:.3f}, Val AUC: {val_auc:.3f}")

    return model


@torch.no_grad()
def eval_link_predictor(model, data):

    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()

    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

In [24]:
def edge_index_coarsen(C_n,theta,labels,graph):
    from torch_geometric.data import Data
    C_n =C_n.detach().cpu()
    theta = theta.detach().cpu()
    L_c= C_n.T@theta@C_n
    from scipy.sparse import csr_matrix
    Wc=(-1*L_c)*(1-np.eye(L_c.shape[0]))
    Wc[Wc<0.01]=0
    Wc= csr_matrix(Wc)
    Wc = Wc.tocoo()
    row = torch.from_numpy(Wc.row).to(torch.long)
    col = torch.from_numpy(Wc.col).to(torch.long)
    edge_index_coarsen2 = torch.stack([row, col], dim=0)
#     print("Edge_index_coarsen Shape", edge_index_coarsen2.shape)
    return edge_index_coarsen2.shape[1]

In [25]:
def link_prediction_auc(C_n,theta,labels,graph):
    from torch_geometric.data import Data
    C_n =C_n.detach().cpu()
    theta = theta.detach().cpu()
    L_c= C_n.T@theta@C_n
    from scipy.sparse import csr_matrix
    Wc=(-1*L_c)*(1-np.eye(L_c.shape[0]))
    Wc[Wc<0.001]=0
    Wc= csr_matrix(Wc)
    Wc = Wc.tocoo()
    row = torch.from_numpy(Wc.row).to(torch.long)
    col = torch.from_numpy(Wc.col).to(torch.long)
    edge_index_coarsen2 = torch.stack([row, col], dim=0)
    print("Edge_index_coarsen Shape", edge_index_coarsen2.shape)
    
        
    edge_weight = torch.from_numpy(Wc.data)
    def one_hot(x, class_count):
        return torch.eye(class_count)[x, :]

    Y = labels
    Y = one_hot(Y,NO_OF_CLASSES)
    P=np.linalg.pinv(C_n)
    labels_coarse = torch.argmax(torch.sparse.mm(torch.Tensor(P).double() , Y.double()).double() , 1)
    data =Data(x=graph , edge_index= edge_index_coarsen2, y=labels_coarse)
    graph = data
    import torch_geometric.transforms as T

    split = T.RandomLinkSplit(
        num_val=0.01,
        num_test=0,
        is_undirected=True,
        add_negative_train_samples=False,
        neg_sampling_ratio=1.0,
    )
    train_data, val_data, test_data = split(graph)
    train_data = train_data.cpu()
    test_data = test_data.cpu()
    val_data = val_data.cpu()

    test_data=tes
    model = Net1(dataset.num_features, 128, 64)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
    criterion = torch.nn.BCEWithLogitsLoss()
    model = train_link_predictor(model, train_data, val_data, optimizer, criterion)

    test_auc = eval_link_predictor(model, test_data)
    print(f"Test: {test_auc:.3f}")
    acc = test_auc*100
    acc

In [ ]:

highest_accuracy=0
for lambda_param in [100,10,1,0.1,0.01,0.001]:
  for beta_param in [0.001,0.01,0.1,1,10,100]:
      for gamma_param in [100,10,1,0.1,0.01,0.001]:
        feat_reduced,C_tilde = experiment_sparsity_2(lambda_param,beta_param,gamma_param,C,theta,X)
        edge_index_c= edge_index_coarsen(C_tilde,theta,labels,feat_reduced)
        print(edge_index_c)
        if edge_index_c >500:
            link_prediction_auc(C_tilde,theta,labels,feat_reduced)
        else:    
            print("breaking")
        

100%|████████████████████████████████████████| 20/20 [1:01:59<00:00, 185.99s/it]


shape of feat torch.Size([9858, 500])
89738
Edge_index_coarsen Shape torch.Size([2, 12840408])
